# Model 3 Version 2.3.2.4
## Isolating trials of learning -- 7 during learning, 7 after learning
### Removing last 3 volumes using FSL ROI

In [1]:
%matplotlib inline
import shutil 
import os
from os.path import join, split, basename
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from glob import glob
from pylab import *

In [37]:
#cope01_FX_before_COND_incorr_run1_trl1_onset5.72.nii.gz
def onset_sort(x):
    x = x.split('_')[-1]
    return(x[5:-8])

subs = ['WMAZE_010']
'''
subs = ['WMAZE_002', 'WMAZE_004', 'WMAZE_005', 'WMAZE_006',
        'WMAZE_007', 'WMAZE_008', 'WMAZE_009', 'WMAZE_010', 'WMAZE_012',
        'WMAZE_017', 'WMAZE_018', 'WMAZE_019', 'WMAZE_020', 'WMAZE_021',
        'WMAZE_022', 'WMAZE_023', 'WMAZE_024', 'WMAZE_026', 'WMAZE_027']
'''

sets = ['set1', 'set2', 'set3']
runs = ['run1', 'run2', 'run3', 'run4', 'run5', 'run6']


for sub in subs:
    sub_dir = '/home/data/madlab/data/mri/wmaze/'
    
    #grab the derivative files created from the set-based learning analysis
    frst_deriv_files = glob(join(sub_dir, 
                                 'scanner_behav/{0}/Bprime_pmode_set*.txt'.format(sub))) 
    frst_deriv_files.sort() 
    
    #grab the learning curve file for B trials
    learning_files = glob(join(sub_dir, 'scanner_behav/{0}/B_pmode_set*.txt'.format(sub))) 
    learning_files.sort()
    
    #grab the 95% confidence file for B trials
    upper_95_files = glob(join(sub_dir, 
                               'scanner_behav/{0}/B_p95_set*.txt'.format(sub)))
    upper_95_files.sort()
    
    #grab all cope files created by LSS Model 5
    cope_files = glob(join(sub_dir, 
                          'frstlvl/wmaze_MRthesis/fixed_before_conditional/model3_2-3-2/{0}/'.format(sub),
                          'modelfit/contrasts/_estimate_model*/cope*_FX_before_COND_*corr_run*_trl*.nii.gz'))     
    cope_files.sort()
    #print cope_files
    
    #grab the behavioral files for all runs
    behav_runs = glob(join(sub_dir, 
                           'scanner_behav/{0}/{0}_wmazebl_2015*.txt'.format(sub))) 
    behav_runs.sort()
    '''
    os.makedirs(join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',
                     'fixed_before_conditional/model3_2-3-2-4/11-11window/{0}/learning_window/'.format(sub)))
    
    os.makedirs(join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',
                     'fixed_before_conditional/model3_2-3-2-4/11-11window/{0}/after_window/'.format(sub)))
    '''
    
    #### LOADING AND ORGANIZING THE COPE FILES ####
    all_runs = []
    for curr_run in runs:
        #selects only the cope files containing the current run's number
        curr_run_files = np.array([f for f in cope_files if curr_run in f])
        #gets the onset time out of the file names using function
        onset_nums = [float(onset_sort(f)) for f in curr_run_files]
        sorted_nums = np.argsort(onset_nums)
        #arranges the actual files according to onset time
        curr_run_files = curr_run_files[sorted_nums]
        all_runs.append(curr_run_files)
        
    
    for i, curr_set in enumerate(sets):
        #load derivative, learning, and p95 files
        deriv_file = np.loadtxt(frst_deriv_files[i])
        learning_curve = np.loadtxt(learning_files[i]) 
        upper_95 = np.loadtxt(upper_95_files[i])
        
        #### COPE FILES ####
        #merge the two runs into one array for the current stim set
        curr_set_copes = np.concatenate((all_runs[i*2], all_runs[i*2+1]))
        #print len(curr_set_copes) 
        
        
        #### GETTING THE DERIV FILES TO MATCH NUMBER OF COPES ####
        #load behavioral files
        behav_run1 = pd.read_table(behav_runs[i*2])
        behav_run2 = pd.read_table(behav_runs[i*2+1])
        
        
        #info concerning onset time
        behav_os = np.empty(320, dtype=object)
        behav_os[:160] = behav_run1['StimOnset'].values
        behav_os[160:] = behav_run2['StimOnset'].values
        
        #info concerning subject response
        behav_resp = np.empty(320, dtype=object)
        behav_resp[:160] = behav_run1['Resp'].values
        behav_resp[160:] = behav_run2['Resp'].values
        
        #info concerning trial type
        behav_type = np.empty(320, dtype=object)
        behav_type[:160] = behav_run1['TrialType'].values
        behav_type[160:] = behav_run2['TrialType'].values
        
        trial_shift = behav_type[:-1] 
        trial_shift = np.insert(trial_shift, 0, -1)
        
        all_b_indices = np.where((behav_type == 'B'))[0]
        #indices of all Bs in the original dataset without preceeding BL trials
        b_BL_indices = np.where((behav_type == 'B') & (trial_shift == 'BL'))[0]
        #grabs only B without preceeding BL trials
        #b_wo_BL_indices = np.where((behav_type == 'B') & (trial_shift != 'BL'))[0]
        #print "# Bs before BLs", len(b_BL_indices)
        #print "# Bs w/o BLs", b_wo_BL_indices       
        
        #isolate bad Bs for removal in learning curve/derivative/p95 files
        bad_Bs = [] 
        bad_Bs.extend(b_BL_indices)
        for curr_B in all_b_indices:
            if not curr_B in bad_Bs:
                #identify in B trials which are non-response
                if behav_resp[curr_B] == 'NR': 
                    bad_Bs.append(curr_B)
                #indices if B trial comes first (observed on 001 run 6)    
                if curr_B in [0, 157, 158, 159, 160, 317, 318, 319]: 
                    bad_Bs.append(curr_B)
        bad_Bs.sort() 
        #print bad_Bs
        
        #get the indices for the bad Bs within the group of Bs
        bad_B_ind = [j for j, f in enumerate(all_b_indices) if f in bad_Bs] 
        bad_B_ind.sort()
        #print bad_B_ind
        #reverse order of Bs to be removed
        bad_B_ind = bad_Bs[::-1]
        print bad_B_ind
      
        #LEARNING CURVE FILES
        #create a temp version of learning_curve
        temp2 = list(learning_curve)
        #pop out the bad Bs starting from the end
        for curr_bad_B in bad_B_ind:
            temp2.pop(curr_bad_B)
        #save without the removed Bs    
        new_learning = np.array(temp2)
        #print len(new_learning)
        alt_learning_curve = new_learning[10:-17]
        
        #P95 FILES
        #create a temp version of upper_95
        temp3 = list(upper_95)
        #pop out the bad Bs starting from the end
        for curr_bad_B in bad_B_ind:
            temp3.pop(curr_bad_B)
        #save without the removed Bs    
        new_upper_95 = np.array(temp3)
        alt_upper_95 = new_upper_95[10:-17]
        #print len(new_upper_95)
        
        # Condition to select subjects and sets that exhibit learning
        if np.mean(new_learning[-10:]) > np.mean(new_upper_95[:10]):
            #DERIV FILES
            #create a temp version of deriv_file
            temp = list(deriv_file)
            #pop out the bad Bs starting from the end
            for curr_bad_B in bad_B_ind:
                temp.pop(curr_bad_B)
            #save the deriv_file without the removed Bs    
            new_deriv = np.array(temp)
            alt_deriv_file = new_deriv[10:-17]
            
            #target_trial = np.argmax(new_deriv)
            #grabs the index of the largest derivative
            alt_target_trial = np.argmax(alt_deriv_file)+10
            
            
            #print len(good_Bs)
            #print good_Bs
        
            #print behav_os[good_Bs-1]
            #print "# copes", len(curr_set_copes)
            #print curr_set_copes
            #convert original behavioral file indices to new B-specific index
            new_indices_B = []
            for n, curr_new_B in enumerate(good_Bs):
                new_indices_B.append(n)
            new_indices_B = np.array(new_indices_B) 
            #print alt_target_trial        
            #print new_indices_B
            #grabs the 7 B trials surrounding the target trial
            b_learning = new_indices_B[alt_target_trial-5:alt_target_trial+6]
            #print b_learning
            #grabs the 7 B trials after learning block
            b_after = new_indices_B[alt_target_trial+6:alt_target_trial+17]
            #print b_after
            #fixed_learning_files = curr_set_copes[b_learning]
            #fixed_after_files = curr_set_copes[b_after] 
            '''
            #copy and save selected learning files to new folder for merge script
            for curr_learning in fixed_learning_files:
                learning_basename =  os.path.basename(curr_learning)  
                shutil.copy2(curr_learning,
                             join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',
                                  'fixed_before_conditional/model3_2-3-2-4/11-11window/{0}/'.format(sub),
                                  'learning_window/{0}'.format(learning_basename)))
               
            #copy and save selected after files to new folder for merge script   
            for curr_after in fixed_after_files:
                after_basename =  os.path.basename(curr_after)
                shutil.copy2(curr_after,
                             join('/home/data/madlab/data/mri/wmaze/frstlvl/wmaze_MRthesis/',
                                  'fixed_before_conditional/model3_2-3-2-4/11-11window/{0}/'.format(sub),
                                  'after_window/{0}'.format(after_basename)))
            '''

[9, 16, 28, 37, 48, 51, 64, 86, 94, 104, 110, 121, 136, 139, 152, 158, 174, 177, 202, 207, 221, 236, 248, 272, 280, 290, 304, 315, 318]
[318, 315, 304, 290, 280, 272, 248, 236, 221, 207, 202, 177, 174, 158, 152, 139, 136, 121, 110, 104, 94, 86, 64, 51, 48, 37, 28, 16, 9]


IndexError: pop index out of range